# Dataset preparation

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import numpy as np
from torchvision import transforms
import src.dataloading as data
from tqdm import tqdm
from sklearn import svm, metrics

# Load data

In [2]:
train_dataloader, valid_dataloader, test_dataloader = data.load_data()

# Model creation

In [3]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Running model on:",device)

Running model on: cuda:0


In [4]:
model = torchvision.models.resnet50(pretrained=True)
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

## Freeze model weights

In [5]:
for param in model.parameters():
    param.requires_grad = False

## loss and optimizer

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

## Trainig

In [7]:
def train(n_epochs=5):
    model.train()
    for epoch in tqdm(range(n_epochs)):
        #train_loss = []
        for data_t, targets_t in train_dataloader:
            data_t = data_t.to(device)
            targets_t = targets_t.to(device)
            optimizer.zero_grad()
            # Generate predictions
            out = model(data_t)
            # Calculate loss
            loss = criterion(out, targets_t)
            #train_loss.append(loss.item())
            # Backpropagation
            loss.backward()
            # Update model parameters
            optimizer.step()
        #print("Loss: ",np.average(train_loss))

## TOP-K ACCURACY

In [8]:
def topk_accuracy(output: torch.Tensor, target: torch.Tensor, topk=(1,)) -> list([torch.FloatTensor]):
    """
    Computes the accuracy over the k top predictions for the specified values of k
    In top-5 accuracy you give yourself credit for having the right answer
    if the right answer appears in your top five guesses.

    ref:
    - https://pytorch.org/docs/stable/generated/torch.topk.html
    - https://discuss.pytorch.org/t/imagenet-example-accuracy-calculation/7840
    - https://gist.github.com/weiaicunzai/2a5ae6eac6712c70bde0630f3e76b77b
    - https://discuss.pytorch.org/t/top-k-error-calculation/48815/2
    - https://stackoverflow.com/questions/59474987/how-to-get-top-k-accuracy-in-semantic-segmentation-using-pytorch

    :param output: output is the prediction of the model e.g. scores, logits, raw y_pred before normalization or getting classes
    :param target: target is the truth
    :param topk: tuple of topk's to compute e.g. (1, 2, 5) computes top 1, top 2 and top 5.
    """
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)
        _, y_pred = output.topk(k=maxk, dim=1)
        y_pred = y_pred.t()
        target_reshaped = target.view(1, -1).expand_as(y_pred)
        correct = (y_pred == target_reshaped)

        list_topk_accs = []
        for k in topk:
            ind_which_topk_matched_truth = correct[:k]
            flattened_indicator_which_topk_matched_truth = ind_which_topk_matched_truth.reshape(-1).float()
            tot_correct_topk = flattened_indicator_which_topk_matched_truth.float().sum(dim=0, keepdim=True)
            topk_acc = tot_correct_topk / batch_size
            list_topk_accs.append(topk_acc.item())
        return list_topk_accs

## Validation

In [9]:
def test(dataloader=test_dataloader):
    val_loss = []
    val_acc = []
    top_1_2_5 = []
    batch_loss = 0
    correct_t = 0
    total_t = 0
    with torch.no_grad():
            model.eval()
            for data_t, target_t in (dataloader):
                data_t, target_t = data_t.to(device), target_t.to(device)
                out = model(data_t)
                loss = criterion(out, target_t)
                top_1_2_5.append(topk_accuracy(out,target_t,(1,2,5)))
                total_t += target_t.size(0)
                val_loss.append(loss.item())
            top_1_2_5 = np.mean(top_1_2_5,axis=0)
            print(f'Loss: {np.mean(val_loss):.4f}, TOP_1: {(top_1_2_5[0]):.3f}, TOP_2: {(top_1_2_5[1]):.3f}, TOP_5: {(top_1_2_5[2]):.3f}\n')

# Replace last fully connected layer

In [10]:
model.fc = nn.Linear(in_features=2048, out_features=120,bias=True)


# Update loss and optimizer for new model

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
model.to(device)
print()

In [12]:
train(10)
torch.save(model.state_dict(), "models/m10_dict.pt")

100%|██████████| 10/10 [09:44<00:00, 58.48s/it]


In [48]:
test(valid_dataloader)

100%|██████████| 10/10 [18:06<00:00, 108.69s/it]


Loss: 0.5682, TOP_1: 0.847%, TOP_2: 0.944%, TOP_5: 0.987%



## Replace classifier layer with SVM

### Remove last fc layer from the model

In [13]:
model_no_fc = torch.nn.Sequential(*(list(model.children())[:-1]))
print(model_no_fc)
model_no_fc.to(device)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


### Prepare data

In [14]:
svm_data = []
svm_targets = []

for data_t, targets_t in tqdm(train_dataloader):
    data_t = data_t.to(device)
    out = model_no_fc(data_t)
    svm_data += [[j.item() for j in i] for i in out.cpu()]
    svm_targets += list(targets_t.numpy())

100%|██████████| 1029/1029 [07:44<00:00,  2.22it/s]


### Create, train and test SVMs with various kernels, with different misclassification allowance

In [15]:
def test_svm(_svm):
    loss = 0
    with torch.no_grad():
        model_no_fc.eval()
        for data_t, target_t in tqdm(valid_dataloader):
            data_t = data_t.to(device)
            out_no_fc = model_no_fc(data_t)
            out_no_fc = [[j.item() for j in i] for i in out_no_fc.cpu()]
            out = _svm.predict(out_no_fc)
            loss += metrics.hamming_loss(y_true=target_t.numpy(), y_pred=out)
        loss /= len(test_dataloader)
        print(f"Accuracy: {(100 * (1-loss)):.4f}%")

In [16]:
for kernel in "linear", "poly", "rbf":
    print("Kernel:", kernel)
    clf = svm.SVC(kernel=kernel, degree=2) # Degree works only for poly kernel.
    for C in 0.001, 0.01, 0.1, 1, 10, 100, 1000:
        print(f"\nC = {C}\n")
        clf.fit(svm_data, svm_targets)
        test_svm(clf)

Kernel: linear

C = 0.001



100%|██████████| 129/129 [01:26<00:00,  1.49it/s]


Accuracy: 80.9205%

C = 0.01



100%|██████████| 129/129 [00:59<00:00,  2.18it/s]


Accuracy: 80.9787%

C = 0.1



...

Accuracy: 84.5930%

C = 100



100%|██████████| 129/129 [01:26<00:00,  1.48it/s]


Accuracy: 84.5349%

C = 1000



100%|██████████| 129/129 [01:26<00:00,  1.49it/s]

Accuracy: 84.5058%


# Unfreeeze last conv block

In [ ]:

for param in model.layer4[2].parameters():
    param.requires_grad = True

In [ ]:
train(10)
test(valid_dataloader)

100%|██████████| 10/10 [09:15<00:00, 55.52s/it]
Loss: 0.4718, TOP_1: 0.851%, TOP_2: 0.945%, TOP_5: 0.986%



# Unfreeeze second-to-last conv block

In [51]:
for param in model.layer4[1].parameters():
    param.requires_grad = True

In [52]:
train(10)
test(valid_dataloader)

100%|██████████| 10/10 [10:06<00:00, 60.63s/it]
Loss: 0.5764, TOP_1: 0.820%, TOP_2: 0.929%, TOP_5: 0.977%



# Testing on unseen data

In [53]:
test(test_dataloader)

Loss: 0.5040, TOP_1: 0.845%, TOP_2: 0.940%, TOP_5: 0.986%



# Untrained model

In [ ]:
model = torchvision.models.resnet50(pretrained=False)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()
model.to(device)
print()

In [ ]:
train(30)
test(valid_dataloader)

100%|██████████| 10/10 [48:52<00:00, 293.25s/it] 


100%|██████████| 30/30 [1:32:57<00:00, 185.92s/it]
Loss: 2.5308, TOP_1: 0.369%, TOP_2: 0.505%, TOP_5: 0.681%



In [10]:
torch.save(model, "models/m10.pt")

In [58]:
torch.save(model, "models/fresh_30.pt")

In [59]:
train(15)

100%|██████████| 15/15 [48:00<00:00, 192.02s/it]


In [60]:
test(test_dataloader)

Loss: 2.2664, TOP_1: 0.441%, TOP_2: 0.570%, TOP_5: 0.748%



In [61]:
torch.save(model.state_dict(), "models/fresh_45_dict.pt")